In [3]:
%pip install mediapipe opencv-python --quiet

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import cv2
import numpy as np
import mediapipe as mp
import math

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # First joint (e.g., shoulder)
    b = np.array(b)  # Middle joint (e.g., elbow)
    c = np.array(c)  # Last joint (e.g., wrist)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0 
stage = None
form_feedback = ""

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            cv2.putText(frame, "Camera not available", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 
                        1, (0, 0, 255), 2, cv2.LINE_AA)
            cv2.imshow('Mediapipe Feed', frame)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
            continue
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # Get left and right coordinates
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Check which hand is used for bicep curl
            if left_wrist[1] < right_wrist[1]:  # Left hand curl
                shoulder = left_shoulder
                elbow = left_elbow
                wrist = left_wrist
                hand = "Left"
            else:  # Right hand curl
                shoulder = right_shoulder
                elbow = right_elbow
                wrist = right_wrist
                hand = "Right"
            
            # Calculate curl angle
            angle = calculate_angle(shoulder, elbow, wrist)

            # Curl counter logic
            if angle > 160:
                stage = "down"
            if angle < 30 and stage == 'down':
                stage = "up"
                counter += 1

            # Priority-based form feedback
            if landmarks[mp_pose.PoseLandmark.NOSE.value].visibility < 0.5:
                form_feedback = "Please adjust your camera to show your full body."
            elif abs(left_shoulder[1] - right_shoulder[1]) > 0.05:
                form_feedback = "Keep your shoulders level."
            elif abs(left_elbow[0] - shoulder[0]) > 0.1:
                form_feedback = "Keep your elbows close to your body."
            elif abs(left_shoulder[0] - landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x) > 0.1 or \
                 abs(right_shoulder[0] - landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x) > 0.1:
                form_feedback = "Keep your back straight."
            elif angle > 160:
                form_feedback = "Make sure to fully extend your arm at the bottom."
            else:
                form_feedback = "Good posture!"
        except Exception as e:
            form_feedback = "No person detected."

        # UI Design
        cv2.rectangle(image, (0, 0), (300, 100), (50, 50, 255), -1)  # Updated color to red
        
        # Display rep count
        cv2.putText(image, 'REPS', (10, 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10, 70), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 3, cv2.LINE_AA)
        
        # Display stage
        cv2.putText(image, 'STAGE', (150, 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2, cv2.LINE_AA)
        cv2.putText(image, stage if stage else "", 
                    (150, 70), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 3, cv2.LINE_AA)

        # Display feedback
        cv2.putText(image, form_feedback if form_feedback else "", (10, 120), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2, cv2.LINE_AA)
        
        # Render detections
        if results.pose_landmarks:  
            mp_drawing.draw_landmarks(image,
                                       results.pose_landmarks,
                                       mp_pose.POSE_CONNECTIONS,
                                       mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2),
                                       mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2))               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

C:\Users\anupa\AppData\Roaming\Python\Python312\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
